# Предобработка:

0. Токенизация и удаление мусора (знаки препинания и прочее). Используется mystem, который расширен для удаления комбинаций вида "пробел-кавычка", "скобка-кавычка", так как такое часто встречается в нашем датасете.
1. Лемматизация. При помощи mystem получаем начальные формы слов.
2. Удаление стоп-слов. За основу взят словарь стоп-слов для русского языка из пакета nltk и дополнен вручную словами приветствия и вопросительными местоимениями, которые наиболее часто встречаются в нашем датасете и не несут особого смысла

```bash
#!/bin/bash
# ВАЖНО! Перейдите в папку ipynb и выполните нижеследующие команды

pip install "msgpack-numpy<0.4.4.0"
wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.vec.gz -O ../app/uploads/cc.ru.300.vec.gz
mkdir -p /tmp/fasttext-vectors/word-vectors-v2
python -m spacy init-model ru /tmp/fasttext-vectors/word-vectors-v2 --vectors-loc ../app/uploads/cc.ru.300.vec.gz
```

In [23]:
import csv
import pandas as pd
import spacy
from pymystem3 import Mystem

In [25]:
nlp = spacy.load('/tmp/fasttext-vectors/word-vectors-v2')

# Предобработка

In [27]:
mystem = Mystem()

RUSSIAN_STOPWORDS = ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'что-то', 
'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 
'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 
'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 
'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 
'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 
'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 
'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 
'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 
'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 
'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 
'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 
'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 
'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 
'конечно', 'всю', 'между', 'почему', 'мочь', 'хотеть', 'какие', 'как', 'привет', 
'здравствуйте', 'приветствую', 'сделать', 'узнавать', 'что-нибудь', 'пожалуйста',
]

TRASH = [' ', ',', ', ', ' ?', '? ', '-', '.', '. ', ')', '(', ' (', ') ', '«', '»', ' «', '» ', '!', '?', '\n']


def lemmatize(text):
    lemmas_with_spaces = mystem.lemmatize(text)
    return lemmas_with_spaces


def remove_stop_words(lemmas):
    return [w for w in lemmas if not w in RUSSIAN_STOPWORDS]


def clean_text(text):
    res = text[:]
    
    for word in text:
        if word in TRASH: 
            res.remove(word)

    return res

# load data

In [ ]:
df = pd.read_csv('../app/faq/dataset.csv', usecols=[
    'text_question', 
    'text_answer', 
    'text_title'])
questions_answers_titles = df.to_dict(orient='records')

In [ ]:
data = open('../app/faq/dataset.csv')
questions_answers_dict = {x[0]: x[1] for x in csv.reader(data, delimiter=',', quotechar='"')}

In [ ]:
faqs = {
'Меня уведомили о том, что мили могут «сгореть». Могу ли я сейчас оформить премию?': 'Мили в программе S7 Priority действительны до конца года, в котором был совершен полет или получена услуга, и в течение двух последующих календарных лет. Станьте активным участником нашей программы, и тогда срок действия миль будет продлён ещё на год.Чтобы стать активным участником программы, нужно:Совершить в течение календарного года (с 1 января по 31 декабря) не менее одного полета в качестве участника на регулярном рейсе S7 или авиакомпаний-партнёров по альянсу по тарифу, участвующему в начислении миль.Накопить не менее 5 000 миль (не считая приветственных за активацию карты и миль, начисляемых в рамках акций и специальных предложений Партнера) за использование платёжной карты S7 Priority при оплате товаров или услуг.Помните, что в программе засчитываются только полёты рейсами S7, авиакомпаний-партнёров альянса oneworld, по тарифам, участвующим в начислении миль, и учтённые в S7 Priority. Полёт не подтверждает ваш статус активного участника, если его тариф не участвует в начислении миль или этот полёт учтён в других программах лояльности.Если Вы не выполнили условия для продления срока действия миль, Вы можете обратиться в Сервисный Центр Программы по телефону горячей линии 8 800 100-77-11 и воспользоваться услугой платного продления. Минимально возможная сумма миль для продления — 500 миль. Стоимость минимального пакета для продления миль составляет 250 р. (1 миля=0.50 р.). Операция по продлению миль не может быть отменена, денежные средства не могут быть возвращены.',
'Какие нужно пройти инстанции, какие потребуются документы, чтобы перевезти в самолёте животное?': 'Перед регистрацией на рейс вашему питомцу необходимо пройти ветеринарный контроль в Службе ветеринарного контроля аэропорта вылета:Если вы летите в города России или в страны СНГ, предъявите ветеринарной службе ветеринарный паспорт и ветеринарное свидетельство животного. После проверки животного документов и состояния животного, в ветеринарное свидетельство ставится печать и вам выдается специальный посадочный талон, который необходимо предъявить перевозитьперевозить перевозить перевозить при регистрации на рейс.Если вы летите международным рейсом (кроме стран СНГ), в обмен перевозить на ветеринарное свидетельство вам выдадут перевозить ветеринарный сертификат, который действует в течение 1—3 дней. Для вывоза животного за рубеж, российской таможне необходимо предъявить справку о том, что животное не представляет племенной ценности.Что требуется ещё:контейнер или клетка, соответствующая требованиям к контейнерам для перевозки животного воздушным транспортом,животному нужно сделать прививку против бешенства не ранее чем за год и не позднее чем за месяц до перелёта.Заранее узнайте обо всех требованиях и порядке оформления документов на перевозку животных в Службе ветеринарного контроля аэропорта вылета.Телефон Службы ветеринарного контроля аэропорта Домодедово (Москва): +7 (495) 967-83-96.Телефоны Службы ветеринарного контроля аэропорта Толмачёво (Новосибирск): +7 (383) 216-98-77.Внимание! Не позднее чем за 48 часов до вылета самолёта вам нужно сделать запрос на перевозку животного, позвонив в контактный центр S7 по телефону 8 800 700-0707 (по России бесплатно).Приезжайте в аэропорт заблаговременно, чтобы без спешки и суеты пройти все необходимые формальности. И помните, что пассажир несёт полную ответственность за комфорт и поведение своего питомца во время перелёта.',
'Что запрещено перевозить в самолёте?': 'Огнестрельное оружие, охотничьи патроны, взрывчатые вещества, фейерверки, сигнальные ракеты.Окисленные материалы и перекиси органического происхождения, отбеливатели и комплекты ремонтных инструментов из стекловолокна.Сжатые газы, бутан, кислород, пропан и баллоны аквалангов.Яды, средства от насекомых, гербициды, мышьяк, цианиды, вещества инфекционного характера, бактерии, вирусы.Едкие вещества, ртуть, кислоты, щёлочи, жидкостный элемент батареек.Легковоспламеняющиеся жидкости и твердые материалы, бензин для зажигалок и печей, краска и все виды спичек.Радиоактивные материалы.',
'Восстановить мили.': 'Автоматическое начисление миль происходит, если при бронировании или регистрации на рейс была предъявлена карта Участника или указан ее номер, а также персональные данные (фамилия, дата рождения и номер счёта S7 Priority) в билете полностью совпадают с данными в личном кабинете. Для зачисления миль системе требуется от 3 до 45 дней с момента полёта, в зависимости от аэропорта вылета.Если какой-то из Ваших полётов остался неучтенным, внесите данные о нём в форму [по ссылке](https://www.s7.ru/personal_account/ffpMilesRecovery.dot) Восстановить мили за полёты рейсами S7, совершенные Вами в качестве Участника, можно за предыдущие 12 месяцев. За полёты на рейсах авиакомпаний-партнёров — за 6 месяцев. Восстанавливая мили за полеты на рейсах партнеров, помните, что не все тарифы участвуют в начислении. Проверить это можно на страницах авиакомпаний-партнёров.',
'Что входит в стоимость аренды авто, которую я оплачиваю на сайте?': 'В стоимость аренды входят неограниченный пробег, страховка от ущерба и угона с франшизой, НДС, дорожный сбор и различные дополнительные сборы (городской, аэропортовый и т.д. — в зависимости от страны и места аренды).'
}

arr = list(faqs.keys())
questions = list(questions_answers_dict.keys())

print(questions)

In [ ]:
a = 'привет'
b = 'здравствуй'

a.similarity(b)

# print results

In [ ]:
question = nlp(' '.join(clean_text(remove_stop_words(lemmatize(input('Введите вопрос: \n'))))))
#question = nlp(input('Ваш вопрос: \n'))

print(question)

for_sort = []

for i in questions:
    #print(nlp(i).text, nlp(i).vector_norm)
    #print(question.similarity(nlp(i)), nlp(i).text)
    for_sort.append((question.similarity(nlp(' '.join(clean_text(remove_stop_words(lemmatize(i)))))), nlp(i).text))
    #print(question.similarity(nlp(' '.join(clean_text(remove_stop_words(lemmatize(i)))))))

sorted_by_second = sorted(for_sort, key=lambda tup: tup[0], reverse=True)

print(sorted_by_second[0:5])

a = input('Выберите правильный ответ: ')
print(questions[int(a)] + str(question))
questions = questions[int(a)] + ' ' + str(question)

In [ ]:
%memit donothing(3)

In [ ]:
!pip list

In [ ]:
%lsmagic

In [ ]:
%timeit range(1000)

In [ ]:
print(1)

In [ ]:
%load_ext line_profiler

In [ ]:
a = 'привет'
b = 'здравствуй'

a.similarity(b)